In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from datetime import datetime
import pytz
from scipy.interpolate import griddata
from mpl_toolkits.mplot3d import Axes3D
from scipy.optimize import minimize
from scipy.interpolate import interp1d
from scipy import integrate
from scipy.integrate import quad

In [3]:
# Comprehensive bullish strategy building for GME 
gme = yf.Ticker("GME")

In [4]:
opt_06_21 = gme.option_chain('2024-06-21')
calls_06_21 = opt_06_21.calls
df_calls_06_21 = pd.DataFrame(calls_06_21)

df_calls_06_21

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency
0,GME240621C00002000,2024-06-17 17:43:07+00:00,2.0,23.10,22.80,23.80,-4.40,-15.999999,4,149,11.875003,True,REGULAR,USD
1,GME240621C00003000,2024-06-14 17:03:07+00:00,3.0,25.47,21.80,22.60,0.00,0.000000,20,32,0.000010,True,REGULAR,USD
2,GME240621C00004000,2024-06-13 18:54:58+00:00,4.0,25.60,20.75,21.75,0.00,0.000000,1,15,7.625000,True,REGULAR,USD
3,GME240621C00005000,2024-06-14 14:15:32+00:00,5.0,23.31,19.80,20.75,0.00,0.000000,4,24,6.718752,True,REGULAR,USD
4,GME240621C00006000,2024-06-13 16:38:00+00:00,6.0,21.50,18.80,20.05,0.00,0.000000,1,7,8.476567,True,REGULAR,USD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,GME240621C00105000,2024-06-17 18:19:30+00:00,105.0,0.08,0.07,0.08,-0.25,-75.757576,670,1952,4.968754,False,REGULAR,USD
84,GME240621C00110000,2024-06-17 18:23:50+00:00,110.0,0.05,0.05,0.13,-0.24,-82.758620,511,2535,5.218753,False,REGULAR,USD
85,GME240621C00115000,2024-06-17 18:48:10+00:00,115.0,0.08,0.05,0.08,-0.21,-72.413795,285,1129,5.140629,False,REGULAR,USD
86,GME240621C00120000,2024-06-17 18:48:10+00:00,120.0,0.07,0.06,0.08,-0.21,-75.000000,3069,5050,5.312503,False,REGULAR,USD


In [5]:
puts_06_21 = opt_06_21.puts
df_puts_06_21 = pd.DataFrame(puts_06_21)
df_puts_06_21

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency
0,GME240621P00002000,2024-06-17 17:31:20+00:00,2.0,0.01,0.00,0.01,0.000000,0.000000,101,3492,7.750000,False,REGULAR,USD
1,GME240621P00003000,2024-06-17 13:30:02+00:00,3.0,0.01,0.00,0.01,0.000000,0.000000,8,679,6.500002,False,REGULAR,USD
2,GME240621P00004000,2024-06-10 14:51:30+00:00,4.0,0.01,0.00,0.01,0.000000,0.000000,44,1457,5.750003,False,REGULAR,USD
3,GME240621P00005000,2024-06-12 14:09:50+00:00,5.0,0.01,0.00,0.01,0.000000,0.000000,20,3377,5.000004,False,REGULAR,USD
4,GME240621P00006000,2024-06-12 18:19:00+00:00,6.0,0.01,0.00,0.01,0.000000,0.000000,193,3258,4.500004,False,REGULAR,USD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,GME240621P00105000,2024-06-07 18:37:51+00:00,105.0,79.07,79.00,80.20,0.000000,0.000000,5,6,6.007815,True,REGULAR,USD
84,GME240621P00110000,2024-06-11 19:21:11+00:00,110.0,83.10,83.95,85.20,0.000000,0.000000,10,11,6.070315,True,REGULAR,USD
85,GME240621P00115000,2024-06-12 18:32:07+00:00,115.0,87.50,88.95,90.20,0.000000,0.000000,1,11,6.203127,True,REGULAR,USD
86,GME240621P00120000,2024-06-07 14:11:35+00:00,120.0,86.15,94.20,95.20,0.000000,0.000000,1,3,6.734377,True,REGULAR,USD


In [6]:

current_date = datetime.now(pytz.timezone('US/Eastern'))
current_date = current_date.replace(tzinfo=None)

df_calls_06_21['expiration_date'] = df_calls_06_21['contractSymbol'].str[3:9]
df_calls_06_21['expiration_date'] = pd.to_datetime(df_calls_06_21['expiration_date'], format='%y%m%d')
df_calls_06_21['TTE'] = (df_calls_06_21['expiration_date'] - current_date).dt.days

df_puts_06_21['expiration_date'] = df_puts_06_21['contractSymbol'].str[3:9]
df_puts_06_21['expiration_date'] = pd.to_datetime(df_puts_06_21['expiration_date'], format='%y%m%d')
df_puts_06_21['TTE'] = (df_puts_06_21['expiration_date'] - current_date).dt.days

df_calls_06_21['mid'] = (df_calls_06_21['bid'] + df_calls_06_21['ask']) / 2
df_puts_06_21['mid'] = (df_puts_06_21['bid'] + df_puts_06_21['ask']) / 2

df_calls_06_21['mid'] = df_calls_06_21['mid'].replace(0, np.nan)
df_puts_06_21['mid'] = df_puts_06_21['mid'].replace(0, np.nan)

df_calls_06_21 = df_calls_06_21.dropna(subset=['mid'])
df_puts_06_21 = df_puts_06_21.dropna(subset=['mid'])

df_calls_06_21['moneyness'] = df_calls_06_21['strike'] / df_calls_06_21['lastPrice']
df_puts_06_21['moneyness'] = df_puts_06_21['strike'] / df_puts_06_21['lastPrice']

df_calls_06_21, df_puts_06_21

(        contractSymbol             lastTradeDate  strike  lastPrice    bid  \
 0   GME240621C00002000 2024-06-17 17:43:07+00:00     2.0      23.10  22.80   
 1   GME240621C00003000 2024-06-14 17:03:07+00:00     3.0      25.47  21.80   
 2   GME240621C00004000 2024-06-13 18:54:58+00:00     4.0      25.60  20.75   
 3   GME240621C00005000 2024-06-14 14:15:32+00:00     5.0      23.31  19.80   
 4   GME240621C00006000 2024-06-13 16:38:00+00:00     6.0      21.50  18.80   
 ..                 ...                       ...     ...        ...    ...   
 83  GME240621C00105000 2024-06-17 18:19:30+00:00   105.0       0.08   0.07   
 84  GME240621C00110000 2024-06-17 18:23:50+00:00   110.0       0.05   0.05   
 85  GME240621C00115000 2024-06-17 18:48:10+00:00   115.0       0.08   0.05   
 86  GME240621C00120000 2024-06-17 18:48:10+00:00   120.0       0.07   0.06   
 87  GME240621C00125000 2024-06-17 18:48:10+00:00   125.0       0.07   0.06   
 
       ask  change  percentChange  volume  openInt